# Ejemplo código de busqueda

Se tomara como referencia una cadena que servirá como ejemplo de busqueda, se le aplicará una similitud coseno con la que obtendremos un libro del conjunto. A partir del libro obtenido, se hará uso de topic distribution para obtener libros relacionados al encontrado al principio.

## Importacion de librerias

In [ ]:
!python --version

In [ ]:
!pip install sentence-transformers
!python -m spacy download es_core_news_sm

In [ ]:
import json
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
import spacy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Ajustables

In [ ]:
consulta = "El hombre no es nada. “Nuestra raza humana es solo un accidente trivial en la historia de la creación, escribió H. P. Lovecraft (Providence, Rhode Island, 1890- 1937), el autor ya de culto en cuyas obras el universo cobija la presencia abominable y repulsiva de criaturas omnipotentes y sin edad. Este volumen enfatiza a esos míticos seres cósmicos que han sellado el destino de la humanidad, mientras Cthulhu sueña y espera en la casa de R’lyeh. La obra de Lovecraft es una literatura ritual que produce una devastadora ansiedad y que hace de la repetición un mecanismo implacable. Se trata de una literatura cuyas huellas son imposibles de borrar."

In [ ]:
ruta_libros_tm_e = '/content/drive/MyDrive/ESCOM/8vo Semestre/NLP/librosTopicEmbeddings.csv'

## Implementación

Previamente se debió de haber limpiado y procesado el dataset. La variable `libros` almacena el .csv con los `topic distribution` y los `embeddings`.
Además, se hace uso de `paraphrase-multilingual-mpnet-base-v2` como modelo para genera el embedding de la consulta (entrada).

In [ ]:
libros = pd.read_csv(ruta_libros_tm_e)
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

Normalización de la consulta, se carga spacy en español

In [ ]:
nlp = spacy.load("es_core_news_sm")

def normalizacion (cadena):
  doc = nlp(cadena)
  texto_normalizado = ""
  for token in doc:
    if(not token.is_stop and not token.pos_=='SPACE'):
      texto_normalizado += f" {token.lemma_}"

  return texto_normalizado[1:]

Se crea el embedding de la consulta y se calcula la simulitud coseno y se almacena en el dataframe para posteriormente ordenar el mismo y obtener la instancia más similar

In [ ]:
sentences1 = [normalizacion(consulta)]

embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = libros['Embeddings'].apply(lambda x: torch.tensor(eval(x)))

embeddings2 = torch.stack(embeddings2.tolist())

cosine_scores = util.cos_sim(embeddings1, embeddings2)

libros['Similitud_Coseno'] = cosine_scores[0].cpu().tolist()

libro_seleccionado = libros[libros['Género']=='Terror y Suspenso'].sort_values('Similitud_Coseno', ascending=False).head(1)
libro_seleccionado

A partir del libro obtenido, se extrae el `Topic Distribution` para calcular la distancia euclidiana con respecto a todos los libros del dataset y obtener los 5 más cernos.

In [ ]:
# Definir la distribución objetivo
distribucion_objetivo = libro_seleccionado['Topic Distribution']
#  {0: 0.583121915183577, 1: 0.0833352365882038, 2: 0.08333337663935135, 3: 0.08337779804310773, 4: 0.0833483889066023, 5: 0.08348328463915791}  # Ejemplo de distribución objetivo
print(distribucion_objetivo)
# Función para convertir el diccionario en un arreglo
def dict_a_array(dict_dist, num_topics=6):
    return np.array([dict_dist.get(i, 0) for i in range(num_topics)])

# Función para calcular la distancia euclidiana
def distancia_euclidiana(distribucion1, distribucion2):
    return np.linalg.norm(distribucion1 - distribucion2)

# Aplicar la conversión y calcular la distancia
libros['Distancia'] = libros['Topic Distribution'].apply(lambda x: distancia_euclidiana(dict_a_array(eval(x)), dict_a_array(distribucion_objetivo)))

# Encontrar la fila con la menor distancia
top_5_mas_cercanos = libros[libros['Género']=='Terror y Suspenso'].sort_values('Distancia').head(5)
top_5_mas_cercanos